# imports

In [2]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from ShaniBA.GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import *
from ShaniBA.TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from tunneltoamazondb import getengine
from pandas import concat, read_csv, Series, read_sql
from ShaniBA.PhenotypicData.PhenotypeGenerationFunctions import *
from ShaniBA.SampleLists.SampleFileFunctions import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

# from SampleFileFunctions import *

done1
stop
stop
done1
done
done
stop


In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# Get glucose info from Nastya:

the get_glucoseInfo_fromNastya was copied to PhenotypeGenerationFunctions.py

In [33]:
NastyaPhensBDgrouped, NastyaPhensBDgroupedOLremoved,NastyaSmokingInfoBDgrouped=get_glucoseInfo_fromNastya()

getting interesting continuous phenotypes from table...
grouping glucsoe info by connection ID....
merging with BD info...
grouping info by BD...
remove outliers...
NastyaPhensBDgrouped describe:
              BMI     BodyFat      HbA1C%        Hips  Max_Glucose  \
count  892.000000  887.000000  909.000000  904.000000   929.000000   
mean    26.622670   30.048843    5.462959  104.688187   163.678897   
std      5.177717    9.314041    0.464185   11.977085    36.785429   
min     16.359310   11.481172    4.100000   62.000000   100.000000   
25%     23.050535   22.702578    5.190000   98.000000   139.000000   
50%     25.654372   30.270595    5.400000  103.000000   155.000000   
75%     29.320139   36.370689    5.700000  110.000000   181.000000   
max     50.452738   64.484954    8.000000  300.000000   400.000000   

       Median_Glucose  Min_Glucose       Waist  WakeupGlucose       UserID  
count      929.000000   929.000000  893.000000     920.000000   929.000000  
mean        95.3626

# new phenotype extraction from the db - 140618:

## extract blood and urine info from the DB

the function extract_blood_and_urine_data_fromDB was copied to PhenotypeGenerationFunctions.py

In [ ]:
questWithLookup_Interesting2=extract_blood_and_urine_data_fromDB()

## extract age information:

the function extract_age_DB and its helper function getCorrectYear were copied to PhenotypeGenerationFunctions.py

In [29]:
def getCorrectYear(x):  # helper function for extract_age_DB function
    try:
        y1 = str(x)[-2:]
        y2 = int(y1)
    except:
        try:
            y1 = str(int(float(x)))[-2:]
            y2 = int(y1)
        except:
            y2 = np.nan
    return y2


def extract_age_DB():
    # extract age information from DF:
    print 'extracting age info from DB'
    Age = read_sql('select  UserID, QuestionID, Answer,TimeStamp from pnp.questionnaire WHERE QuestionID=415 or QuestionID=517 ', getengine())
    print 'number of answers is %s' % len(Age)
    
    # extract answer's year:
    print 'extracting answers year'
    for n in Age.index:
        m = re.search('(?<=20)\w+', Age.loc[n, 'TimeStamp'])
        if m is not None:
            Age.loc[n, 'year'] = '20' + m.group(0)
    Age.year=pd.to_numeric(Age.year, errors='coerce')
    Age.year=Age.year.apply(lambda x: getCorrectYear(x))
    Age.Answer=pd.to_numeric(Age.Answer, errors='coerce')
    Age.Answer=Age.Answer.apply(lambda x: getCorrectYear(x))
    print 'years are:'
    print Age.year.unique().tolist()
    print Age.head()
    
    print Age.dtypes
      
    
    print 'extracting the correct age per User...'
    Age = Age.drop_duplicates(subset=['UserID', 'QuestionID'])
    Age['Calculated Age']= np.where(Age['Answer'] > 18, (Age.year+2000) - (Age['Answer']+1900), 
                                                        (Age.year+2000) - (Age['Answer']+2000))
    print 'calculated ages are:'
    print Age['Calculated Age'].value_counts(dropna=False).sort_index()
   

    Age2 = Age.pivot(index='UserID', columns='QuestionID', values='Calculated Age')
    Age2['Age'] = np.where(Age2[415].notnull(), Age2[415], Age2[517])
#     Age2['YOB2'] = Age2['YOB'].apply(lambda x: getCorrectYear(x))
#     Age2['Age'] = np.where(Age2['YOB2'] > 18, (Age2.year+2000) - (Age2['YOB2'] + 1900), 
#                                                (Age2.year - 2000 - Age2['YOB2'])

    print 'Ages are:'
    print Age2['Age'].value_counts(dropna=False).sort_index()
   
    
    
    print 'converting Users to BD samples...'
    
    Age3 = pd.DataFrame(Age2['Age'])
    
    # add BD:
    # load BD SAMPLE FILE:
    f1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/BDfile_31052018.xlsx'          
    BDfile = pd.read_excel(f1)
    BDfile_small = BDfile[['DnaID', 'UserID']]
    BDfile_small = BDfile_small.drop_duplicates()
    BDfile_small = BDfile_small.rename(columns={'DnaID':'BD'})
    Age4 = pd.merge(BDfile_small, Age3, how='left', left_on='UserID', right_index=True)
    
    Age5 = Age4[Age4.Age.notnull()]
    print Age5.head()
    
    print 'the youngest subjects are:'
    print Age5.sort_values(by='Age').head()
    
    print 'the oldest subjects are:'
    print Age5.sort_values(by='Age').tail()
    
    print 'subjects with no indicated age are:'
    print Age5[Age5['Age'].isnull()]
    
    print 'number of BD samples with age is %s' % len(Age5)

    print 'saving into excel file...'
    f1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/NewPhenotypicData/PNPage.xlsx'          
    Age5.to_excel(f1)
    
    print 'done'
    
    return Age5


In [30]:
Age5=extract_age_DB()

extracting age info from DB
number of answers is 3314
extracting answers year
years are:
[13, 14, 15, 16, 17, 18]
   UserID  QuestionID  Answer           TimeStamp  year
0      39         415    78.0   7/1/2013 10:54:12    13
1      40         415    84.0  6/30/2013 14:14:19    13
2      26         415    87.0  6/26/2013 14:19:35    13
3      52         415    91.0   7/7/2013 10:04:57    13
4      22         415    72.0  7/14/2013 12:53:52    13
UserID          int64
QuestionID      int64
Answer        float64
TimeStamp      object
year            int64
dtype: object
extracting the correct age per User...
calculated ages are:
 4.0      1
 5.0      2
 8.0      4
 9.0      1
 10.0     4
 11.0     6
 12.0     6
 13.0     3
 14.0     5
 15.0     7
 17.0     4
 18.0    12
 19.0     9
 20.0     9
 21.0    16
 22.0    11
 23.0    20
 24.0    28
 25.0    36
 26.0    48
 27.0    59
 28.0    66
 29.0    66
 30.0    83
 31.0    73
 32.0    75
 33.0    69
 34.0    77
 35.0    59
 36.0    70
      

## extract gender info.

In [ ]:
the function extract_Gender_DB and its helper function getCorrectYear were copied to PhenotypeGenerationFunctions.py

In [31]:
GenderDF6=extract_Gender_DB()
print len(GenderDF6)
GenderDF6.head()

extracting Gender info from DB
number of answers is 3343
extracting the correct answer per User...
check users with ambigiuos answers:...
QuestionID     414     516    2342  notNone
UserID                                     
60            Male    Male     NaN        2
2146        Female  Female     NaN        2
4567        Female  Female     NaN        2
4873        Female  Female     NaN        2
8123        Female  Female     NaN        2
8367        Female  Female     NaN        2
8397           NaN    male    Male        2
8494           NaN    male    Male        2
8495           NaN    male    Male        2
8496           NaN  female  Female        2
8497           NaN    male    Male        2
8498           NaN  female  Female        2
8499           NaN  female  Female        2
8500           NaN  female  Female        2
8501           NaN    male    Male        2
8502           NaN  female  Female        2
8503           NaN  female  Female        2
8504           NaN    male

,BD,UserID,Gender
0,BD1,2252,Female
1,BD2,1879,Male
2,BD3,2314,Female
3,BD4,2238,Female
4,BD5,2465,Male


## extracting BMI info

In [ ]:
the function extract_BMI_DB and its helper function getCorrectYear were copied to PhenotypeGenerationFunctions.py

In [32]:
BMIgrouped,BMIgroupedOLremoved=extract_BMI_DB()

extracting BMI info from DB...
2675
fixing nonsense height and weight data...
calculating BMI...
adding BD info and converting to BD-based data
n samples after merging is 2708
n samples after nan removal is 1314


/usr/wisdom/python/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/wisdom/python/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


number of BD samples after grouping is 949
number of BD samples with either weight or height info is 942
number of BD samples with BMI info is 887
remove outliers...
BMIgrouped describe:
            UserID      weight      height         BMI
count   942.000000  925.000000  904.000000  887.000000
mean   2437.267516   74.827387  167.431416   26.623058
std    1781.143157   16.539618    9.059282    5.160374
min      12.000000   42.000000  147.000000   16.359310
25%    1112.750000   63.000000  161.000000   23.069812
50%    2064.000000   72.000000  167.000000   25.711662
75%    3614.500000   84.000000  174.000000   29.338726
max    9381.000000  151.000000  194.000000   50.452738
BMIgroupedOLremoved describe:
            UserID      weight      height         BMI
count   942.000000  923.000000  904.000000  883.000000
mean   2437.267516   74.668833  167.431416   26.518107
std    1781.143157   16.201644    9.059282    4.929931
min      12.000000   42.000000  147.000000   16.359310
25%    1112.7

## general function to extract single-question info from the DB:

In [ ]:
the function extract_singleQinfo_DB and its helper function getCorrectYear were copied to PhenotypeGenerationFunctions.py

### extract creatinine info:

In [ ]:
creatinine, creatinineremoved=extract_singleQinfo_DB(QuestionID=1012, QuestionName='Creatinine')

### extract WBC info:

In [ ]:
WBC, WBCremoved=extract_singleQinfo_DB(QuestionID=1038, QuestionName='WBC')
WBC.head()

### extract CRP (WIDE RANGE) info:

In [ ]:
CRP_wr, CRP_wrremoved=extract_singleQinfo_DB(QuestionID=1013, QuestionName='CRP_wr')
CRP_wr.head()

### extract CRP info:

In [ ]:
CRP_hs, CRP_hsremoved=extract_singleQinfo_DB(QuestionID=1014, QuestionName='CRP_hs')
CRP_hs.head()

### extract hemoglobin info:

In [ ]:
Hemoglobin, Hemoglobinremoved=extract_singleQinfo_DB(QuestionID=1018, QuestionName='Hemoglobin')
Hemoglobin.head()

### extract INFO FOR MORE BLOOD COUNT PARAMETERS:

In [ ]:
qList=[('LDH',1084),('AST',1007),('LDL',1086),('HDL',1017),('Triglycerides',1088),('glucose',1045),('Total Cholesterol',1011)]

for q in qList:
    
    QuestionID=q[1]
    QuestionName=q[0]
    print '*******%s*******' %QuestionName

    phen,phenRemoved=extract_singleQinfo_DB(QuestionID=QuestionID, QuestionName=QuestionName)
    print phen.head()

# extract blood pressure data:

In [3]:
f1='%s/TCR_real_data/NewPhenotypicData/BPinfo.csv' %MyPath
BPinfo=pd.read_csv(f1)
BPinfo.head()

,UserID,BP
0,26,148/84
1,39,122/63
2,40,129/76
3,52,107/67
4,33,104/63


In [6]:
f1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/BDfile_31052018.xlsx'          
BDfile = pd.read_excel(f1)
BDfile.head()

,DnaID,storageDT,UserID,corrected_regCode,correction status (reg number),Blood_Year,Blood_Date,UserID_bYear,UserID_bDate
0,BD1,2014-09-10 17:13:12,2252,738601,same,2014,2014-09-10,2252_2014,2252_2014-09-10
1,BD2,2014-09-10 17:12:22,1879,450464,same,2014,2014-09-10,1879_2014,1879_2014-09-10
2,BD3,2014-09-10 17:12:45,2314,127686,same,2014,2014-09-10,2314_2014,2314_2014-09-10
3,BD4,2014-09-17 18:40:53,2238,134681,same,2014,2014-09-17,2238_2014,2238_2014-09-17
4,BD5,2014-09-17 18:44:08,2465,462009,same,2014,2014-09-17,2465_2014,2465_2014-09-17


In [19]:
def merge_with_PNP530Cardio126BD(phenDF,phenDF_colToMerge='UserID',BDfile_colToMerge='UserID'):
    #load PNP530Cardio126 sample list:
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
        PNP530=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
        Cardio126=pickle.load(fp)
    #load BD SAMPLE FILE:
    f1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/BDfile_31052018.xlsx'          
    BDfile = pd.read_excel(f1)
    
    BDfile_small=BDfile[['DnaID',BDfile_colToMerge]]
#     print len(BDfile_small)
    BDfile_small=BDfile_small.drop_duplicates()
    BDfile_small=BDfile_small.rename(columns={'DnaID':'BD'})
    
    PNP530Cardio126regs=BDfile_small[BDfile_small['BD'].isin(Cardio126+PNP530)]
    
    # get the phenotypic data only for pnp530Cardio126 samples and index by BD:
    phen_with_PNP530Cardio126BD=pd.merge(PNP530Cardio126regs,phenDF,how='left', left_on=BDfile_colToMerge,right_on=phenDF_colToMerge)
    phen_with_PNP530Cardio126BD=phen_with_PNP530Cardio126BD.drop(BDfile_colToMerge,axis=1).set_index('BD')
    
    print ('phen_with_PNP530Cardio126BD.shape: ',phen_with_PNP530Cardio126BD.shape)
    if phen_with_PNP530Cardio126BD.shape[0]==656:
        print 'df length is ok'
    else:
        print 'df length is not 656, removing duplicates:'
        print 'those are the duplicates:'
        df=phen_with_PNP530Cardio126BD.reset_index()
        print df[df.duplicated(subset='BD', keep=False)]
#         phen_with_PNP530Cardio126BD = phen_with_PNP530Cardio126BD.reset_index().drop_duplicates(subset='BD', keep='last').set_index('BD')
#         print ('phen_with_PNP530Cardio126BD.shape now is: ',phen_with_PNP530Cardio126BD.shape)
 
    
    return phen_with_PNP530Cardio126BD
        
    
    

In [20]:
BP=merge_with_PNP530Cardio126BD(BPinfo)

('phen_with_PNP530Cardio126BD.shape: ', (883, 1))
df length is not 656, removing duplicates:
those are the duplicates:
        BD       BP
7      BD8      NaN
8      BD8   123/74
9      BD8   126/69
12    BD11   117/77
13    BD11    99/70
14    BD12      NaN
15    BD12   119/75
16    BD12   110/61
49   BD108   101/65
50   BD108      NaN
59   BD123   127/83
60   BD123  135/118
81   BD149      NaN
82   BD149   120/78
83   BD149   120/78
98   BD199   104/60
99   BD199   139/87
115  BD275  147/101
116  BD275   115/75
117  BD275      NaN
118  BD275      NaN
121  BD292   142/76
122  BD292      NaN
123  BD292  128/103
124  BD292  143/101
125  BD292      NaN
126  BD292      NaN
127  BD292  146/106
143  BD339   130/87
144  BD339   120/74
..     ...      ...
660  BD868      NaN
661  BD868   124/76
662  BD868   111/79
663  BD870   130/73
664  BD870      NaN
665  BD870   108/62
666  BD870   111/57
667  BD870      NaN
668  BD870      NaN
669  BD870      NaN
670  BD875      NaN
671  BD875      NaN
6

In [15]:
BP.shape

(883, 1)

In [16]:
BP.head()

,BP
BD,
BD1,115/83
BD2,133/74
BD3,132/75
BD4,122/78
BD5,137/73


# general function to extract multiple-questions info from the db: < not finished >

In [ ]:
# to find questionID, start with:
# lookup = read_sql('select  QuestionID,Question, QuestionType from pnp.questions', getengine())
# search=lookup[lookup.Question.str.contains(questionWord)]
#and examine the questions. then use the following function with the selected questionIDlist

def extract_phenotype_DB(questionIDlist,phenotypeName):
    lookup = read_sql('select  QuestionID,Question, QuestionType from pnp.questions', getengine())
    print 'lookup list length is %s' %len(lookup)
    for n,questionID in enumerate(questionIDlist): 
        print n,questionID
        answers = read_sql('select  UserID, QuestionID, Answer from pnp.questionnaire WHERE QuestionID=%s' %questionID, getengine())
        if n==0:
            answerDF=answers
        else:
            answerDF=pd.concat([answerDF,answers])
            
    return answerDF
        
    
    
#print 'question list length is %s' %len(Age)

In [ ]:
# extract all answers for this question

In [ ]:
# add BDs based on UserIDs

In [ ]:
#merge answers based on BDs: